### 1. Importing libraries

In [14]:
import pandas as pd
import numpy as np
import plotly.express as px
from matplotlib import pyplot as plt
from matplotlib import gridspec
import ipywidgets as widgets
from ipywidgets import fixed
from geopy.geocoders import Nominatim
import geopy.geocoders
import certifi
import ssl


### 2. Loading dataset

In [4]:
ds = pd.read_csv('Datasets/kc_house_data.csv')
df = ds

### 3. Business Problems


1. Qual a média do preço de compra dos imóveis por “Nível”?
- Nível 0 -> Preço entre R$ 0 e R$ 321.950
- Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
- Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
- Nível 3 -> Acima de R$ 645.000
2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft
3. Adicione as seguinte informações ao conjunto de dados original:
- Place ID: Identificação da localização
- OSM Type: Open Street Map type
- Country: Nome do País
- Country Code: Código do País
4. Adicione os seguinte filtros no Mapa:
- Tamanho mínimo da área da sala de estar.
- Número mínimo de banheiros.
- Valor Máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das Condições do Imóvel.
- Filtro por Ano de Construção.
5. Adicione os seguinte filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não.


### 5. Solution

In [5]:
# 1. Qual a média do preço de compra dos imóveis por “Nível”?
#     - Nível 0 -> Preço entre R$ 0 e R$321.950
#     - Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
#     - Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
#     - Nível 3 -> Acima de R$ 645.000

# define level of prices
for i in range( len( df ) ):
    if df.loc[i, 'price'] <= 321950:
        df.loc[i, 'level'] = 0
    elif ( df.loc[i,'price'] > 321950 ) & ( df.loc[i,'price'] <= 450000 ):
        df.loc[i, 'level'] = 1
    elif ( df.loc[i,'price'] > 450000 ) & ( df.loc[i,'price'] <= 645000 ):
        df.loc[i, 'level'] = 2
    else:
        df.loc[i, 'level'] = 3

In [6]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,0.0
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2.0
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,770,0,1933,0,98028,47.7379,-122.233,2720,8062,0.0
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2.0
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2.0


In [10]:
#mean of prices grouped by property level
df[['price', 'level']].groupby('level').mean().reset_index()

,level,price
0,0.0,251557.649889
1,1.0,385688.681502
2,2.0,539730.960007
3,3.0,987540.224456


In [11]:
# 2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
# - Size 0 -> Tamanho entre 0 e 1427 sqft
# - Size 1 -> Tamanho entre 1427 e 1910 sqft
# - Size 2 -> Tamanho entre 1910 e 2550 sqft
# - Size 3 -> Tamanho acima de 2550 sqft

# define sizes
for i in range( len( df ) ):
    if df.loc[i, 'sqft_living'] <= 1427:
        df.loc[i, 'size'] = 0
    elif ( df.loc[i,'sqft_living'] > 1427 ) & ( df.loc[i,'sqft_living'] <= 1910 ):
        df.loc[i, 'size'] = 1
    elif ( df.loc[i,'sqft_living'] > 1910 ) & ( df.loc[i,'sqft_living'] <= 2550 ):
        df.loc[i, 'size'] = 2
    else:
        df.loc[i, 'size'] = 3

In [12]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level,size
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,0,1955,0,98178,47.5112,-122.257,1340,5650,0.0,0.0
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,400,1951,1991,98125,47.7210,-122.319,1690,7639,2.0,3.0
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,0,1933,0,98028,47.7379,-122.233,2720,8062,0.0,0.0
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,910,1965,0,98136,47.5208,-122.393,1360,5000,2.0,2.0
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,0,1987,0,98074,47.6168,-122.045,1800,7503,2.0,1.0


In [13]:
#mean of living room area grouped by property size
df[['sqft_living', 'size']].groupby('size').mean().reset_index()

,size,sqft_living
0,0.0,1123.832531
1,1.0,1664.962334
2,2.0,2211.792786
3,3.0,3329.607016


In [35]:
# 3. Adicione as seguinte informações ao conjunto de dados original:
# - Place ID: Identificação da localização
# - OSM Type: Open Street Map type
# - Country: Nome do País
# - Country Code: Código do País

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
geopy.geocoders.options.default_ssl_context = ctx

geolocator = Nominatim( scheme='https', user_agent='geoapiExercises')

#API response test
response = geolocator.reverse( '47.5112,-122.257' )
print(response.raw['place_id'])
print(response.raw['osm_type'])
print(response.raw['address']['country'])
print(response.raw['address']['country_code'])


161022547
way
United States
us


In [29]:
#create blank columns
df['place_id'] = 'NA'
df['osm_type'] = 'NA'
df['country'] = 'NA'
df['country_code'] = 'NA'

In [31]:
newatt = ['place_id', 'osm_type', 'country', 'country_code']

for i in range (5): #testing loop for 5 lines

    #defining latitude and longitude
      
    latlong = str(df.loc[i, 'lat']) + ',' + str(df.loc[i, 'long'])

    #API request
    response = geolocator.reverse( latlong )

    for att in newatt:
        if att in response.raw['address']:
            df.loc[i, att] = response.raw['address'][att]
        elif att in response.raw:
            df.loc[i, att] = response.raw[att]

In [32]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,level,size,place_id,osm_type,country,country_code
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,47.5112,-122.257,1340,5650,0.0,0.0,161022547,way,United States,us
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,47.7210,-122.319,1690,7639,2.0,3.0,160016678,way,United States,us
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,47.7379,-122.233,2720,8062,0.0,0.0,74951055,node,United States,us
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,47.5208,-122.393,1360,5000,2.0,2.0,156726151,way,United States,us
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,47.6168,-122.045,1800,7503,2.0,1.0,296045104,way,United States,us


In [ ]:
data_map = ds[['id', 'lat', 'long', 'price']]

In [ ]:
mapa = px.scatter_mapbox( data_map, lat='lat', lon='long', hover_name='id', hover_data=['price'], color_discrete_sequence=['darkmagenta'], zoom=8, height=300)
mapa.update_layout( mapbox_style='open-street-map')
mapa.update_layout( height=600, margin={'r':0, 't':0,'l':0,'b':0 })
mapa.show()

In [ ]:
mapa.write_html( 'datasets/mapa_report.html')

### 6. New Business Problems

1. Crie uma nova coluna chamada: “house_age”
    * Se o valor da coluna “date” for maior que 2014-01-01 => ‘new_house’
    * Se o valor da coluna “date” for menor que 2014-01-01 => ‘old_house’
2. Crie uma nova coluna chamada: “dormitory_type”
    * Se o valor da coluna “bedrooms” for igual à 1 => ‘studio’
    * Se o valor da coluna “bedrooms” for igual a 2 => ‘apartament’
    * Se o valor da coluna “bedrooms” for maior que 2 => ‘house’
3. Crie uma nova coluna chamada: “condition_type”
    * Se o valor da coluna “condition” for menor ou igual à 2 => ‘bad’
    * Se o valor da coluna “condition” for igual à 3 ou 4 => ‘regular’
    * Se o valor da coluna “condition” for igual à 5 => ‘good’
4. Modifique o TIPO a Coluna “condition” para STRING
5. Delete as colunas: “sqft_living15” e “sqft_lot15”
6. Modifique o TIPO a Coluna “yr_build” para DATE
7. Modifique o TIPO a Coluna “yr_renovated” para DATE
8. Qual a data mais antiga de construção de um imóvel?
9. Qual a data mais antiga de renovação de um imóvel?
10. Quantos imóveis tem 2 andares?
11. Quantos imóveis estão com a condição igual a “regular” ?
12. Quantos imóveis estão com a condição igual a “bad”e possuem “vista para água” ?
13. Quantos imóveis estão com a condição igual a “good” e são “new_house”?
14. Qual o valor do imóvel mais caro do tipo “studio” ?
15. Quantos imóveis do tipo “apartment” foram reformados em 2015 ?
16. Qual o maior número de quartos que um imóveis do tipo “house” possui ?
17. Quantos imóveis “new_house” foram reformados no ano de 2014?
18. Selecione as colunas: “id”, “date”, “price”, “floors”, “zipcode” pelo método:
    * Direto pelo nome das colunas.
    * Pelos Índices.
    * Pelos Índices das linhas e o nome das colunas
    * Índices Booleanos
19. Salve um arquivo .csv com somente as colunas do item 10 ao 17.
20. Modifique a cor dos pontos no mapa de “pink” para “verde-escuro”

In [ ]:
#1. Crie uma nova coluna chamada: “house_age”
    #Se o valor da coluna “date” for maior que 2014-01-01 => ‘new_house’
    #Se o valor da coluna “date” for menor que 2014-01-01 => ‘old_house’

ds['house_age'] = 'old_house'

date_criteria = pd.to_datetime('2014-01-01', format='%Y-%m-%d')
ds.loc[ds['date'] > date_criteria, 'house_age'] = 'new_house'


In [ ]:
# 2. Crie uma nova coluna chamada: “dormitory_type”
    # Se o valor da coluna “bedrooms” for igual à 1 => ‘studio’
    # Se o valor da coluna “bedrooms” for igual a 2 => ‘apartament’
    # Se o valor da coluna “bedrooms” for maior que 2 => ‘house’

ds['dormitory_type'] = ds['bedrooms'].apply( lambda x: 'studio' if x ==1 else 'apartment' if x ==2 else 'house')

In [ ]:
# 3. Crie uma nova coluna chamada: “condition_type”
    # Se o valor da coluna “condition” for menor ou igual à 2 => ‘bad’
    # Se o valor da coluna “condition” for igual à 3 ou 4 => ‘regular’
    # Se o valor da coluna “condition” for igual à 5 => ‘good’

ds['condition_type'] = ds['condition'].apply( lambda x: 'bad' if x <= 2 else 'regular' if x <= 4 else 'good')


In [ ]:
#4. Modifique o TIPO a Coluna “condition” para STRING

ds['condition'] = ds['condition'].astype('str')


In [ ]:
#5. Delete as colunas: “sqft_living15” e “sqft_lot15”

ds = ds.drop(['sqft_living15' , 'sqft_lot15'], axis=1)
ds.columns

In [ ]:
#6. Modifique o TIPO a Coluna “yr_build” para DATE

ds['yr_built'] = pd.to_datetime(ds['yr_built'], format='%Y')

In [ ]:
#7. Modifique o TIPO a Coluna “yr_renovated” para DATE

ds['yr_renovated'] = ds['yr_renovated'].apply( lambda x: pd.to_datetime( '1900-01-01', format='%Y-%m-%d') if x == 0 else pd.to_datetime( x, format='%Y') )

In [ ]:
#8. Qual a data mais antiga de construção de um imóvel?

oldest_built = ds[['id', 'yr_built']].sort_values('yr_built', ascending=True).reset_index().loc[0, 'yr_built']
oldest_built

# or
# ds['yr_built'].min()


In [ ]:
#9. Qual a data mais antiga de renovação de um imóvel?

ds.loc[ds['yr_renovated'] > pd.to_datetime('1900-01-01', format='%Y-%m-%d'),'yr_renovated'].min()

# oldest_renovated = ds[['id', 'yr_renovated']].sort_values('yr_renovated', ascending=False).reset_index().loc[0, 'yr_renovated']
# oldest_renovated

In [ ]:
#10. Quantos imóveis tem 2 andares?

len(ds.loc[ds['floors'] == 2, :])



In [ ]:
#11. Quantos imóveis estão com a condição igual a “regular” ?

len(ds.loc[ds['condition_type'] == 'regular', :])


In [ ]:
#12. Quantos imóveis estão com a condição igual a “bad”e possuem “vista para água” ?

len(ds[(ds['condition_type'] == 'bad') & (ds['waterfront'] == 1)])

In [ ]:
#13. Quantos imóveis estão com a condição igual a “good” e são “new_house”?

len(ds[(ds['condition_type'] == 'good') & (ds['house_age'] == 'new_house')])


In [ ]:
#14. Qual o valor do imóvel mais caro do tipo “studio” ?

studio_expensive = ds[(ds['dormitory_type'] == 'studio')].sort_values('price', ascending=False).reset_index().loc[0, 'price']
studio_expensive


In [ ]:
# 15. Quantos imóveis do tipo “apartment” foram reformados em 2015 ?

len(ds[(ds['dormitory_type'] == 'apartment') & (ds['yr_renovated'] == pd.to_datetime('2015-01-01'))])

In [ ]:
# 16. Qual o maior número de quartos que um imóveis do tipo “house” possui ?

ds[(ds['dormitory_type'] == 'house')].sort_values('bedrooms', ascending=False).reset_index().loc[0, 'bedrooms']

In [ ]:
# 17. Quantos imóveis “new_house” foram reformados no ano de 2014?

len(ds[(ds['house_age'] == 'new_house') & (ds['yr_renovated'] == pd.to_datetime('2014-01-01'))])

In [ ]:
# 18. Selecione as colunas: “id”, “date”, “price”, “floors”, “zipcode” pelo método:
#     * Direto pelo nome das colunas.
#     * Pelos Índices.
#     * Pelos Índices das linhas e o nome das colunas

ds.columns

In [ ]:
ds[['id', 'date', 'price', 'floors', 'zipcode']]

In [ ]:
ds.iloc[:, [0,1,2,7,16]]

In [ ]:
ds.loc[:, ['id', 'date', 'price', 'floors', 'zipcode']]

In [ ]:
# 19. Salve um arquivo .csv com somente as colunas do item 10 ao 17.
ds.columns

In [ ]:
ds.iloc[:,10:18].to_csv( 'exercicio18.csv')

In [ ]:
# 20. Modifique a cor dos pontos no mapa de “pink” para “verde-escuro”

mapa = px.scatter_mapbox( data_map, lat='lat', lon='long', hover_name='id', hover_data=['price'], color_discrete_sequence=['darkgreen'], zoom=8, height=300)
mapa.update_layout( mapbox_style='open-street-map')
mapa.update_layout( height=600, margin={'r':0, 't':0,'l':0,'b':0 })
mapa.show()

In [ ]:
df_grouped = ds[['id','bedrooms']].groupby('bedrooms')

for bedrooms, frame in df_grouped:
    print('Number of bedrooms: {}'.format(bedrooms))
    print( frame.shape[0])



In [ ]:
# 1. Qual o número de imóveis por ano de construção?

ds[['id','yr_built']].groupby('yr_built').count()

# for year, frame in df_grouped:
#     print('Number of properties: {}'.format(year))
#     print( frame.shape[0])

In [ ]:
# 2. Qual o menor número de quartos por ano de construção dos imóveis?

ds[['bedrooms','yr_built']].groupby('yr_built').min()

In [ ]:
# 3. Qual o preço de compra mais alto por cada número de quartos?
pd.set_option('display.float_format', lambda x: '%.2f' % x)
ds[['bedrooms','price']].groupby('bedrooms').max()

In [ ]:
# 4. Qual a soma de todos os preços de compra por cada número de quartos?

df = ds[['bedrooms','price']].groupby('bedrooms').sum()
df.round(2)

In [ ]:
# 5. Qual a soma de todosos preços de compra pelo número de quartos e banheiros?
df = ds[['bedrooms','price', 'bathrooms']].groupby(['bedrooms', 'bathrooms']).sum()
df

In [ ]:
# 6. Qual o tamanho médio das salas dos imóveis por ano de construção?

df = ds[['sqft_living','yr_built']].groupby('yr_built').mean()
df

In [ ]:
ds.columns

In [ ]:
# 7. Qual o tamanho mediano das salas dos imóveis por ano de construção?

df = ds[['sqft_living','yr_built']].groupby('yr_built').median()
df

In [ ]:
# 8. Qual o desvio-padrãodo tamanho das salas dos imóveis por ano de construção?

df = ds[['sqft_living','yr_built']].groupby('yr_built').std()
df

In [ ]:
# 9. Como é o crescimento médio dos preços de compra dos imóveis, por dia e semana do ano?

ds['year'] = ds['yr_built'].dt.year
by_year = ds[['price', 'year']].groupby('year').sum().reset_index()
by_year

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.bar( by_year['year'], by_year['price'])

In [ ]:
from matplotlib import gridspec
#
fig = plt.figure ( figsize=(20,12) )
specs = gridspec.GridSpec( ncols=2, nrows=2, figure=fig )
#
ax1 = fig.add_subplot( specs[0, :]) #First Row
ax2 = fig.add_subplot( specs[1, 0]) #Second Row, First Column
ax3 = fig.add_subplot( specs[1, 1]) #Second Row, Second Column
#
#First Graph
#
#
df=ds
df['year'] = pd.to_datetime( df['yr_built'] ).dt.year
by_year = df[['id','year']].groupby( 'year' ).mean().reset_index()
#plt.figure( figsize=(20,10))
ax1.bar( by_year['year'],by_year['id'] )
#
#Second Graph
#
df['day'] = pd.to_datetime( df['date'] )
by_day = df[['id','day']].groupby( 'day' ).mean().reset_index()
#by_day
#plt.figure( figsize=(20,12))
ax2.plot( by_day['day'],by_day['id'] )

# Thrid Graph
#
df['year_week'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%U')
by_week_of_year = df[['id','year_week']].groupby( 'year_week' ).mean().reset_index()
#plt.figure( figsize=(20,10))
ax3.plot( by_week_of_year['year_week'],by_week_of_year['id'] )
plt.xticks( rotation=60 );


In [ ]:
# 10. Eu gostaria de olharno mapa e conseguir identificar as casas com  maior preço.

mapa = px.scatter_mapbox( data_map, lat='lat', lon='long', size='price', hover_name='id', hover_data=['price'], color_discrete_sequence=['darkgreen'], zoom=9, height=300)
mapa.update_layout( mapbox_style='open-street-map')
mapa.update_layout( height=600, margin={'r':0, 't':0,'l':0,'b':0 })
mapa.show()

2. Faça um gráfico de barras que represente a soma dos preços pelo número de quartos.
3. Faça um gráfico de linhas que represente a média dos preços pelo ano construção dos imóveis.
4. Faça um gráfico de barras que represente a média dos preços pelo tipo dos dormitórios.
5. Faça um gráfico de linha que mostre a evolução da média dos preços pelo ano da reforma dos
imóveis, a partir do ano de 1930.
6. Faça um tabela que mostre a média dos preços por ano de construção e tipo de dormitórios dos imóveis.
7. Crie um Dashboard com os gráficos das questões 02, 03, 04 ( Dashboard: 1 Linha e 2 colunas)
8. Crie um Dashboard com os gráficos das perguntas 02, 04 ( Dashboard: 2 colunas )
9. Crie um Dashboard com os gráficos das perguntas 03, 05 ( Dashboard: 2 Linhas )
10. Faça um gráfico com o tamanho dos pontos sendo igual ao tamanho da sala de estar

In [ ]:
# 2. Faça um gráfico de barras que represente a soma dos preços pelo número de quartos.

df=ds
by_bedrooms = df[['price','bedrooms']].groupby( 'bedrooms' ).sum().reset_index()
#by_bedrooms
plt.figure( figsize=(20,10))
graph2 = plt.bar( by_bedrooms['bedrooms'],by_bedrooms['price'] )
graph2

In [ ]:
# 3. Faça um gráfico de linhas que represente a média dos preços pelo ano construção dos imóveis.

df=ds
by_yr_built = df[['price','yr_built']].groupby( 'yr_built' ).mean().reset_index()
#by_yr_built
plt.figure( figsize=(20,10))
graph3 =plt.plot( by_yr_built['yr_built'],by_yr_built['price'] )
graph3

In [ ]:
# 4. Faça um gráfico de barras que represente a média dos preços pelo tipo dos dormitórios.

df=ds
by_dormitory_type = df[['price','dormitory_type']].groupby( 'dormitory_type' ).mean().reset_index()
by_dormitory_type
plt.figure( figsize=(20,12))
plt.bar( by_dormitory_type['dormitory_type'],by_dormitory_type['price'] )


In [ ]:
# 5. Faça um gráfico de linha que mostre a evolução da média dos preços pelo ano da reforma dos imóveis, a partir do ano de 1930.

df=ds
by_yr_renovated = df[['price','yr_renovated']].groupby( 'yr_renovated' ).mean().reset_index()
#by_yr_renovated
plt.figure( figsize=(20,10))
plt.plot( by_yr_renovated['yr_renovated'],by_yr_renovated['price'] )


In [ ]:
# 6. Faça um tabela que mostre a média dos preços por ano de construção e tipo de dormitórios dos imóveis.

df=ds
mean_prices_by_yr_built_dormitory_type = df[['price','yr_built','dormitory_type']].groupby(['yr_built','dormitory_type'] ).mean().reset_index()
mean_prices_by_yr_built_dormitory_type

#by_yr_renovated

In [ ]:
# 7. Crie um Dashboard com os gráficos das questões 02, 03, 04 ( Dashboard: 1 Linha e 2 colunas)

fig = plt.figure ( figsize=(20,12) )
specs = gridspec.GridSpec( ncols=2, nrows=2, figure=fig )
#
ax1 = fig.add_subplot( specs[0, :]) #First Row
ax2 = fig.add_subplot( specs[1, 0]) #Second Row, First Column
ax3 = fig.add_subplot( specs[1, 1]) #Second Row, Second Column
#
#First Graph
#
#
df = ds
by_bedrooms = df[['price','bedrooms']].groupby( 'bedrooms' ).sum().reset_index()
ax1.bar( by_bedrooms['bedrooms'],by_bedrooms['price'] )
#
#Second Graph
#
by_yr_built = df[['price','yr_built']].groupby( 'yr_built' ).mean().reset_index()
ax2.plot( by_yr_built['yr_built'],by_yr_built['price'] )

# Thrid Graph
#
by_dormitory_type = df[['price','dormitory_type']].groupby( 'dormitory_type' ).mean().reset_index()
ax3.bar( by_dormitory_type['dormitory_type'],by_dormitory_type['price'] )


In [ ]:
# 8. Crie um Dashboard com os gráficos das perguntas 02, 04 ( Dashboard: 2 colunas )

fig = plt.figure ( figsize=(20,12) )
specs = gridspec.GridSpec( ncols=2, nrows=1, figure=fig )
#
ax1 = fig.add_subplot( specs[0, 0]) #First, First Column
ax2 = fig.add_subplot( specs[0, 1]) #First Row, Second Column
#
#First Graph
#
#
df = ds
by_bedrooms = df[['price','bedrooms']].groupby( 'bedrooms' ).sum().reset_index()
ax1.bar( by_bedrooms['bedrooms'],by_bedrooms['price'] )
#
#Second Graph
#
by_dormitory_type = df[['price','dormitory_type']].groupby( 'dormitory_type' ).mean().reset_index()
ax2.bar( by_dormitory_type['dormitory_type'],by_dormitory_type['price'] )

In [ ]:
# 9. Crie um Dashboard com os gráficos das perguntas 03, 05 ( Dashboard: 2 Linhas )

fig = plt.figure ( figsize=(20,12) )
specs = gridspec.GridSpec( ncols=1, nrows=2, figure=fig )
#
ax1 = fig.add_subplot( specs[0, 0]) #First Row
ax2 = fig.add_subplot( specs[1, 0]) #Second Row
#
#First Graph
#
df=ds
by_yr_built = df[['price','yr_built']].groupby( 'yr_built' ).mean().reset_index()
ax1.plot( by_yr_built['yr_built'],by_yr_built['price'] )
#
#Second Graph
#
by_yr_renovated = df[['price','yr_renovated']].groupby( 'yr_renovated' ).mean().reset_index()
ax2.plot( by_yr_renovated['yr_renovated'],by_yr_renovated['price'] )

In [ ]:
# 10. Faça um gráfico com o tamanho dos pontos sendo igual ao tamanho da sala de estar

data_map = ds[['id', 'lat', 'long', 'price', 'sqft_living']]

mapa = px.scatter_mapbox( data_map, lat='lat', lon='long', size='sqft_living', hover_name='id', hover_data=['price'], color_discrete_sequence=['darkgreen'], zoom=9, height=300)
mapa.update_layout( mapbox_style='open-street-map')
mapa.update_layout( height=600, margin={'r':0, 't':0,'l':0,'b':0 })
mapa.show()